In [3]:
import os
from tqdm import tqdm
from datasets import load_dataset
import sys
sys.path.append('../evaluation')
from evaluate import RetrievalSystem, main as evaluate_main
import yaml
import hyde

None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.


In [4]:
with open("/users/christineye/retrieval/config.yaml", 'r') as stream:
    api_key = yaml.safe_load(stream)['openai_api_key']

In [2]:
%load_ext autoreload
%autoreload 2

In [5]:
from vector_store import EmbeddingClient, Document, DocumentLoader

In [11]:
test_hyde = hyde.HydeRetrievalSystem(embeddings_path = "/users/christineye/retrieval/data/vector_store/embeddings_matrix.npy",
                         documents_path = "/users/christineye/retrieval/data/vector_store/documents.pkl",
                         index_mapping_path = "/users/christineye/retrieval/data/vector_store/index_mapping.pkl", config_path = "/users/christineye/retrieval/config.yaml", 
                                     generate_n = 3, embed_query = True, max_doclen = 100)

Loading embeddings...
Loading documents...
Loading index mapping...
Processing document dates...
Loading metadata...
Data loaded successfully.


In [21]:
query = "What is the stellar mass of the Milky Way?"
arxiv_id = "2301.00001"
top_k = 10
results = test_hyde.retrieve(query, arxiv_id, top_k)

In [9]:
import weighted
test_weighted = weighted.WeightedRetrievalSystem()

Loading embeddings...
Loading documents...
Loading index mapping...
Processing document dates...
Data loaded successfully.
Loading existing index...
Index loaded successfully.


In [10]:
test_weighted.make_bow()
test_weighted.make_embed()

Loading existing index...
Index loaded successfully.
Loading embeddings...
Loading documents...
Loading index mapping...
Processing document dates...
Data loaded successfully.


In [52]:
query = "What is the stellar mass of the Milky Way?"
arxiv_id = "2301.00001"
top_k = 10
results = test_weighted.retrieve(query, arxiv_id, top_k)

In [11]:
astro_meta = load_dataset("JSALT2024-Astro-LLMs/astro_paper_corpus", split = "train")

Generating train split: 100%|██████████| 271544/271544 [00:05<00:00, 54228.25 examples/s]


In [18]:
weighted.bow.preprocess_text(astro_meta[0]['keyword_search'])

AttributeError: module 'weighted' has no attribute 'bow'